In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from models.addmul import HandleAddMul

In [3]:
network_cache_dir = "networks/cache-networks/"
network_name = "lyr256-split0.8-lr0.01-add-mul.data"

checkpoint = True
test_flag = 1

input_dims = [42]
output_dims = [20]
batchsize = 128
tau = 1  # temperature parameter, NB: check for value in paper

handler = HandleAddMul(input_dims, output_dims, dir=network_cache_dir + network_name, checkpoint=checkpoint, lr=0.001)

... FNN Network training on cuda:0 ...
Accessing : networks/cache-networks/lyr256-split0.8-lr0.01-add-mul.data
networks/cache-networks/lyr256-split0.8-lr0.01-add-mul.data
Load saves ...


# Test the Add mask

In [4]:
# Freeze Parameters in Network
for name, param in handler.network.named_parameters():
    param.requires_grad = False
# Fetch Mask
load_logits = torch.load('trained_logits_mul_mask.pt')

binary_mask = []
for layer in load_logits:
    binary_mask.append((layer > 0.5).float())


In [5]:
train_split = 0.8
test_split = 1 - train_split

data_fp = ["generate_datasets/tmp/digit-data/simple_add.npy",
           "generate_datasets/tmp/digit-data/simple_mul.npy"]
data_add = np.load(data_fp[0], allow_pickle=True)
data_mul = np.load(data_fp[1], allow_pickle=True)
data_add_len = len(data_add)
data_mul_len = len(data_mul)

train_split_idx_add = int(data_add_len * train_split)
train_split_idx_mul = int(data_mul_len * train_split)
test_data_add = data_add[train_split_idx_add:]
test_data_mul = data_mul[train_split_idx_mul:]

test_loader_add = torch.utils.data.DataLoader(dataset=torch.Tensor(test_data_add), batch_size=batchsize, shuffle=True)
test_loader_mul = torch.utils.data.DataLoader(dataset=torch.Tensor(test_data_mul), batch_size=batchsize, shuffle=True)

def cycle(iterable):
    while True:
        for x in iterable:
            yield x

iterator_test_add = iter(cycle(test_loader_add))
iterator_test_mul = iter(cycle(test_loader_mul))

In [8]:
loss_hist = []
NUM_EPOCHS = 100

mul_acc = 0
add_acc = 0

for e in range(NUM_EPOCHS):
    print(f'Starting epoch {e}...')

    '''Call Addition Mask'''
    iterator_binaries = iter(binary_mask)
    batch_add = next(iterator_test_add)
    batch_mul = next(iterator_test_mul)
    
    batches = [batch_add, batch_mul]
    
    with torch.no_grad():
        for layer in handler.network.layers[0]:
            if isinstance(layer, torch.nn.Linear):
                layer.weight.data * next(iterator_binaries)

        '''Pass Testing batches through the network'''
        res = [] 
        for batch in batches:
            # Load in batch data (not binaries for one-hot input)
            inp = torch.stack([torch.stack([b[0], b[1]]) for b in batch])
            otp = torch.stack([b[2] for b in batch])
            ops = torch.stack([b[3] for b in batch])
            # Convert batch data toone-hot representation
            inp, _ = handler.set_batched_digits(inp, otp, ops)

            inp_ = inp.to(handler.network.device)
            
            otp = otp.to(handler.network.device)

            otp_pred = handler.network(inp_)
            
            otp_pred = torch.stack([otp_pred[:,:10], otp_pred[:,10:]], dim=1)
            otp_pred = torch.argmax(otp_pred, dim=2)
            otp_pred = otp_pred[:, 0]*10 +otp_pred[:, 1]
            res.append([otp_pred, otp])
    
        for i, results in enumerate(res):
            
            diff = results[1] - results[0]
            running_acc = len(diff[diff == 0])/(diff.size()[0])
            if not i:
                add_acc += running_acc
            else:
                mul_acc += running_acc

print(f'Add Accuracy: {add_acc/NUM_EPOCHS} \n Mul Accuracy: {mul_acc/NUM_EPOCHS}')

Starting epoch 0...
[tensor([[ 7.3095e-04, -4.4802e-03, -2.3847e-03,  ...,  2.1170e-03,
          8.7902e-03,  1.4896e-03],
        [-1.0086e-03,  1.4704e-03,  1.1490e-03,  ...,  7.3990e-04,
         -2.5133e-03, -2.0569e-03],
        [ 2.5322e-03,  1.9542e-03,  1.3701e-03,  ...,  8.2235e-03,
          1.4239e-03,  3.2492e-03],
        ...,
        [ 2.9260e-03,  1.8464e-03,  1.7536e-03,  ...,  1.1334e-02,
          9.9348e-01, -3.9156e-03],
        [ 2.6197e-03,  2.0726e-03,  6.4772e-04,  ..., -9.5427e-05,
          1.8222e-03, -1.3306e-03],
        [-7.0691e-04, -1.0636e-03, -6.1502e-04,  ..., -1.2702e-03,
         -6.5522e-04, -3.1897e-03]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]
[tensor([[ 0.0009, -0.0007,  0.0016,  ..., -0.0032,  0.0019, -0.0009],
   

[tensor([[-1.3315e-03,  1.5493e-03,  2.0651e-04,  ..., -4.1188e-03,
         -1.5118e-03,  3.2195e-03],
        [ 7.3992e-03,  1.8448e-04, -3.6462e-04,  ...,  4.4853e-03,
          2.5079e-03, -1.7937e-03],
        [-2.1428e-03,  1.2298e-03, -5.9585e-04,  ..., -2.3978e-03,
         -5.3437e-03,  5.7298e-03],
        ...,
        [ 5.8081e-04,  1.1638e-03,  4.1042e-04,  ..., -2.0758e-03,
         -1.6489e-03,  1.0455e-03],
        [ 3.7493e-03, -9.0450e-06, -9.8068e-04,  ...,  2.9152e-03,
          6.6880e-05,  1.0006e+00],
        [-7.3580e-04,  6.7815e-04, -6.8766e-03,  ..., -2.9615e-03,
          1.0526e-02, -8.3573e-04]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]])]
Starting epoch 5...
[tensor([[ 6.4053e-04,  2.8553e-04,  9.7677e-04,  ...,  2.1259e-03,
      

[tensor([[ 1.4938e-03, -1.4327e-04, -1.1085e-03,  ...,  9.0178e-04,
          1.4170e-03, -9.5758e-04],
        [ 2.5986e-03, -2.2118e-03,  5.4421e-03,  ...,  7.9689e-03,
         -8.8302e-03,  1.0093e+00],
        [ 3.4875e-03, -2.8340e-03, -2.0895e-03,  ...,  3.5300e-03,
         -1.8815e-03, -2.7428e-03],
        ...,
        [-4.8377e-04,  3.2336e-06, -1.5538e-03,  ..., -3.8251e-03,
         -2.4140e-04,  1.0068e+00],
        [ 2.9260e-03,  1.8464e-03,  1.7536e-03,  ...,  1.1334e-02,
          9.9348e-01, -3.9156e-03],
        [-1.3794e-03, -5.5744e-04, -1.1660e-03,  ..., -4.9327e-04,
          1.1133e-04, -1.0453e-03]], device='cuda:0'), tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]
Starting epoch 9...
[tensor([[ 7.3150e-05, -7.1079e-06, -1.7082e-03,  ..., -1.9236e-03,
      

[tensor([[ 3.9914e-03,  9.9245e-04, -1.6543e-04,  ...,  4.9663e-04,
          2.5183e-03,  1.4210e-03],
        [ 3.3596e-03,  2.5165e-03,  2.8425e-03,  ..., -1.9955e-04,
          1.0002e+00,  3.3208e-03],
        [-4.6790e-04, -1.6863e-03,  1.4142e-03,  ..., -1.3818e-03,
          2.3292e-04, -5.5449e-03],
        ...,
        [-5.8291e-03, -1.4909e-03,  1.9129e-03,  ..., -1.7782e-03,
         -3.9365e-03, -4.0264e-03],
        [ 2.8160e-03,  2.1785e-04,  2.0424e-04,  ...,  1.7982e-03,
          9.9279e-01, -2.4221e-03],
        [-8.6369e-04,  1.9023e-03,  3.5139e-04,  ...,  8.9101e-04,
         -6.6022e-04, -1.4074e-03]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]
[tensor([[-1.6678e-03,  1.2558e-03,  1.9362e-03,  ...,  2.3344e-03,
          9.9969e-01, -1.0

[tensor([[ 3.1452e-03,  5.6273e-04,  3.4335e-04,  ...,  1.9389e-03,
          1.7978e-05,  9.9594e-01],
        [ 1.0478e-02, -2.0562e-03, -9.9981e-04,  ..., -4.3741e-03,
         -3.1745e-03,  1.0747e-03],
        [-5.3845e-05, -2.2433e-03, -4.6998e-04,  ...,  1.0248e+00,
         -3.5563e-03, -1.6500e-03],
        ...,
        [ 2.1355e-03, -5.7695e-04, -7.2868e-04,  ..., -6.9719e-04,
          2.1564e-03,  3.1496e-03],
        [-9.1451e-04,  5.3309e-05, -1.1252e-03,  ...,  1.6758e-03,
         -2.5165e-04,  1.2054e-04],
        [ 4.4337e-04, -7.9281e-04, -5.3713e-04,  ...,  2.6094e-03,
         -8.5603e-04,  1.0050e+00]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])]
[tensor([[-2.3752e-03,  3.1808e-03,  2.1123e-04,  ...,  6.4847e-03,
          1.6326e-03,  3.0

...

[tensor([[ 5.8081e-04,  1.1638e-03,  4.1042e-04,  ..., -2.0758e-03,
         -1.6489e-03,  1.0455e-03],
        [-1.6678e-03,  1.2558e-03,  1.9362e-03,  ...,  2.3344e-03,
          9.9969e-01, -1.0698e-03],
        [ 9.4869e-04,  9.6395e-04,  1.6313e-03,  ...,  2.6729e-04,
          7.4541e-03,  9.8397e-01],
        ...,
        [ 2.5879e-04,  1.4543e-03,  5.4947e-04,  ...,  1.4991e-03,
         -1.5347e-04, -7.6772e-04],
        [ 1.9721e-03,  2.7656e-04,  3.5521e-04,  ...,  4.6573e-03,
          1.1097e-03, -2.8714e-03],
        [-1.6835e-03,  1.4860e-03,  7.0319e-04,  ..., -6.7134e-03,
         -1.1698e-03, -4.0666e-04]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])]
Starting epoch 83...
[tensor([[ 3.0474e-03,  1.8053e-03,  1.8376e-03,  ...,  1.0052e+00,
     

[tensor([[ 0.0064,  0.0025,  0.0012,  ...,  0.0109,  0.0005,  0.0001],
        [ 0.0006, -0.0035, -0.0029,  ...,  0.0067, -0.0041,  0.0007],
        [ 0.0017,  0.0021,  0.0013,  ...,  0.0015, -0.0006, -0.0019],
        ...,
        [ 0.0022,  0.0024,  0.0014,  ..., -0.0003, -0.0026,  0.0007],
        [-0.0063,  0.0062, -0.0045,  ..., -0.0050,  0.0056, -0.0108],
        [ 0.0002,  0.0003,  0.0001,  ...,  0.0073,  0.0011, -0.0042]],
       device='cuda:0'), tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])]
Starting epoch 87...
[tensor([[ 2.7933e-03,  3.5962e-04,  2.3712e-03,  ...,  2.3706e-03,
         -3.0684e-04,  9.8938e-01],
        [ 1.9966e-03, -1.1446e-03,  2.5195e-04,  ..., -5.0605e-04,
         -2.3803e-03,  3.1183e-04],
        [ 5.2108e-04,  1.0452e-03, -2.1285e-03,  ...,  2.

[tensor([[-0.0004, -0.0014, -0.0042,  ..., -0.0004,  0.0024, -0.0013],
        [ 0.0015,  0.0010,  0.0011,  ...,  0.0016,  0.0017,  0.0020],
        [ 0.0011,  0.0023, -0.0011,  ...,  0.0008,  0.0034,  0.0025],
        ...,
        [ 0.0023,  0.0003,  0.0011,  ..., -0.0020,  0.0011,  0.0008],
        [ 0.0009, -0.0007,  0.0016,  ..., -0.0032,  0.0019, -0.0009],
        [ 0.0008, -0.0015,  0.0003,  ...,  0.0008, -0.0005,  0.0010]],
       device='cuda:0'), tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])]
Starting epoch 91...
[tensor([[ 1.2756e-03, -3.0895e-04, -3.2874e-04,  ...,  6.4110e-03,
          1.0059e+00, -4.6864e-04],
        [-1.2178e-03,  4.9076e-03, -6.3133e-04,  ...,  9.8652e-01,
          4.3898e-03,  4.9364e-03],
        [-1.1344e-05,  1.1574e-04, -8.2136e-04,  ...,  1.

[tensor([[-3.4122e-03,  2.7590e-03,  1.8904e-03,  ...,  1.0081e+00,
         -9.4177e-04,  3.4464e-03],
        [-4.4088e-04,  4.3061e-04, -1.7089e-03,  ..., -5.2847e-03,
         -5.1565e-05,  1.0078e+00],
        [ 1.8326e-03, -1.3922e-04,  6.5739e-04,  ...,  5.0930e-03,
          1.2539e-03, -1.3636e-03],
        ...,
        [ 7.9022e-03,  1.7826e-02,  3.3152e-03,  ...,  2.9415e-02,
          9.3648e-01, -1.1095e-02],
        [-4.4288e-04, -7.2884e-03,  2.7032e-03,  ..., -1.7063e-02,
         -6.3893e-03,  2.5253e-03],
        [-5.6960e-03,  4.7300e-03, -5.6066e-03,  ..., -7.4984e-03,
          1.6430e-02,  7.8690e-03]], device='cuda:0'), tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]
Starting epoch 95...
[tensor([[ 2.6336e-04, -3.0023e-03, -2.0345e-03,  ...,  8.0124e-04,
     

[tensor([[-6.5248e-05,  1.3133e-04, -3.4267e-04,  ..., -2.1532e-04,
          1.2323e-03, -2.1165e-03],
        [ 1.6319e-03, -3.6517e-03,  4.2515e-03,  ...,  8.8258e-04,
         -8.9397e-03,  1.0235e+00],
        [ 2.0036e-03,  1.8153e-03,  4.0375e-04,  ..., -2.7709e-03,
          3.6962e-03, -1.3085e-03],
        ...,
        [ 1.3539e-03,  2.4428e-03,  1.0169e-03,  ...,  7.9177e-03,
          2.1824e-03,  1.2486e-03],
        [ 1.2065e-03,  5.1927e-04,  4.8126e-04,  ..., -7.9220e-03,
          1.0499e-02, -6.6096e-03],
        [-2.4557e-03,  8.4968e-03,  1.4578e-03,  ...,  1.4301e-03,
         -3.4345e-03, -3.4352e-04]], device='cuda:0'), tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])]
Starting epoch 99...
[tensor([[ 4.1034e-04, -5.4855e-04,  1.6128e-03,  ..., -1.4054e-03,
     